In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
from transformers import T5Config, TFT5Model, T5Model, load_tf_weights_in_t5, T5Tokenizer
from transformers import T5ForConditionalGeneration, TFT5ForConditionalGeneration

/home/ubuntu/.local/lib/python3.8/site-packages/apex/pyprof/__init__.py:5: FutureWarning: pyprof will be removed by the end of June, 2022
  warnings.warn("pyprof will be removed by the end of June, 2022", FutureWarning)


In [3]:
!rm -rf temp2

In [4]:
import os

out = 'temp2'
os.makedirs(out, exist_ok=True)

In [5]:
config = T5Config.from_pretrained('malay-huggingface/t5-super-tiny-bahasa-cased')
config.save_pretrained(out)

In [6]:
!ls t5-super-tiny-noisy-en-ms

checkpoint
events.out.tfevents.1659118087.husein-MS-7D31
events.out.tfevents.1659118188.husein-MS-7D31
events.out.tfevents.1659119840.husein-MS-7D31
graph.pbtxt
model.ckpt-1015000.data-00000-of-00002
model.ckpt-1015000.data-00001-of-00002
model.ckpt-1015000.index
model.ckpt-1015000.meta
model.ckpt-1047000.data-00000-of-00002
model.ckpt-1047000.data-00001-of-00002
model.ckpt-1047000.index
model.ckpt-1047000.meta
model.ckpt-1080000.data-00000-of-00002
model.ckpt-1080000.data-00001-of-00002
model.ckpt-1080000.index
model.ckpt-1080000.meta
model.ckpt-1112000.data-00000-of-00002
model.ckpt-1112000.data-00001-of-00002
model.ckpt-1112000.index
model.ckpt-1112000.meta
model.ckpt-1145000.data-00000-of-00002
model.ckpt-1145000.data-00001-of-00002
model.ckpt-1145000.index
model.ckpt-1145000.meta
model.ckpt-1177000.data-00000-of-00002
model.ckpt-1177000.data-00001-of-00002
model.ckpt-1177000.index
model.ckpt-1177000.meta
model.ckpt-1197000.data-00000-of-00002
model.ckpt-1197000.data-00001-of-00002

In [7]:
model = T5Model(config)
load_tf_weights_in_t5(model, config, 't5-super-tiny-noisy-en-ms/model.ckpt-1201000')

T5Model(
  (shared): Embedding(32128, 256)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 256)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=256, out_features=384, bias=False)
              (k): Linear(in_features=256, out_features=384, bias=False)
              (v): Linear(in_features=256, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=256, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): FusedRMSNorm(torch.Size([256]), eps=1e-06, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=256, out_features=1024, bias=False)
              (wo): Linear(in_features=1024, out_features=256, bias=F

In [8]:
tokenizer = T5Tokenizer('sp10m.cased.ms-en.model', padding = True)

In [9]:
model.save_pretrained(out)

In [10]:
model_gen = T5ForConditionalGeneration.from_pretrained(out)

In [11]:
model_tf = TFT5ForConditionalGeneration.from_pretrained(out, from_pt = True)

2022-08-01 15:04:16.736034: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-01 15:04:16.740027: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-01 15:04:16.740688: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-01 15:04:16.741772: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [12]:
with open('en-ms-right.test') as fopen:
    right = fopen.read().split('\n')
    
with open('en-ms-left.test') as fopen:
    left = fopen.read().split('\n')

In [13]:
model_gen.cuda()

T5ForConditionalGeneration(
  (shared): Embedding(32128, 256)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 256)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=256, out_features=384, bias=False)
              (k): Linear(in_features=256, out_features=384, bias=False)
              (v): Linear(in_features=256, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=256, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): FusedRMSNorm(torch.Size([256]), eps=1e-06, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=256, out_features=1024, bias=False)
              (wo): Linear(in_features=1024, out_f

In [14]:
from tqdm import tqdm

batch_size = 1

results = []
for i in tqdm(range(0, len(left), batch_size)):
    input_ids = [{'input_ids': tokenizer.encode(f'terjemah Inggeris ke Melayu: {s}', return_tensors = 'pt')[0]} for s in left[i:i + batch_size]]
    padded = tokenizer.pad(input_ids, padding = 'longest')
    outputs = model_gen.generate(padded['input_ids'].cuda(), attention_mask = padded['attention_mask'].cuda(), max_length = 1000)
    for o in outputs:
        results.append(tokenizer.decode(o, skip_special_tokens=True))

100%|████████████████████████████████████████████████████████████████████████████████████| 69744/69744 [2:40:19<00:00,  7.25it/s]


In [15]:
# from tqdm import tqdm

# batch_size = 2

# results = []
# for i in tqdm(range(0, len(left[:1]), batch_size)):
#     input_ids = [{'input_ids': tokenizer.encode(f'terjemah Inggeris ke Melayu: {s}', return_tensors = 'pt')[0]} for s in left[i:i + batch_size]]
#     padded = tokenizer.pad(input_ids, padding = 'longest')
#     outputs = model_gen.generate(**padded, max_length = 1000)
#     for o in outputs:
#         results.append(tokenizer.decode(o, skip_special_tokens=True))

In [16]:
results[0]

'Anda tahu bagaimana kisahnya berjalan. Dua orang heteroseksual, hampir pasti orang kulit putih bergelut dalam hidup mereka. Mereka saling bertemu, berkumpul, berpisah, bersatu lagi dan mencari kebahagiaan dan penyelesaian yang benar di antara satu sama lain senjata. Akhirnya, mereka dapat memulakan hidup normal! Rom-com boleh menikmati eskapisme yang menyeronokkan, tetapi kisah yang terlalu sering diceritakan, yang terlalu banyak bergantung pada kepelbagaian, terlalu bergantung pada stereotaip jantina dan terlalu mementingkan penjualan kami jenama cinta yang tidak mungkin dapat hidup hingga: sebuah kajian di Heriot Watt University mendapati bahawa rom-com mempunyai kesan negatif terhadap hubungan, menjadikan kita mengejar standard cinta yang tidak dapat dicapai. Dalam proses menulis drama baru saya, Ross & Rachel, yang menghadapi mitos cinta moden dan terbuka di Festival Edinburgh Fringe Ogos ini, saya harus banyak memikirkan percintaan dalam fiksyen dari Romeo dan Juliet hingga Pride

In [17]:
from sacrebleu.metrics import BLEU, CHRF, TER

bleu = BLEU()

In [18]:
filtered_left, filtered_right = [], []
for no, r in enumerate(results):
    if len(r):
        filtered_left.append(r)
        filtered_right.append(right[no])

In [19]:
refs = [filtered_right]
sys = filtered_left

In [20]:
sys[0]

'Anda tahu bagaimana kisahnya berjalan. Dua orang heteroseksual, hampir pasti orang kulit putih bergelut dalam hidup mereka. Mereka saling bertemu, berkumpul, berpisah, bersatu lagi dan mencari kebahagiaan dan penyelesaian yang benar di antara satu sama lain senjata. Akhirnya, mereka dapat memulakan hidup normal! Rom-com boleh menikmati eskapisme yang menyeronokkan, tetapi kisah yang terlalu sering diceritakan, yang terlalu banyak bergantung pada kepelbagaian, terlalu bergantung pada stereotaip jantina dan terlalu mementingkan penjualan kami jenama cinta yang tidak mungkin dapat hidup hingga: sebuah kajian di Heriot Watt University mendapati bahawa rom-com mempunyai kesan negatif terhadap hubungan, menjadikan kita mengejar standard cinta yang tidak dapat dicapai. Dalam proses menulis drama baru saya, Ross & Rachel, yang menghadapi mitos cinta moden dan terbuka di Festival Edinburgh Fringe Ogos ini, saya harus banyak memikirkan percintaan dalam fiksyen dari Romeo dan Juliet hingga Pride

In [21]:
# for i in range(len(filtered_left)):
#     print(filtered_left[i])
#     print(filtered_right[i])
#     print(right[i])
#     print()

In [22]:
# filtered_right

In [23]:
# filtered_left

In [24]:
r = bleu.corpus_score(sys, refs)

In [27]:
r.__dict__

{'name': 'BLEU',
 'score': 58.72114029430599,
 '_mean': -1.0,
 '_ci': -1.0,
 '_verbose': '80.4/64.0/53.0/44.7 (BP = 0.994 ratio = 0.994 hyp_len = 2614280 ref_len = 2630014)',
 'bp': 0.9939995916897871,
 'counts': [2102642, 1627808, 1311221, 1074639],
 'totals': [2614280, 2544537, 2474822, 2405299],
 'sys_len': 2614280,
 'ref_len': 2630014,
 'precisions': [80.42910476307053,
  63.972659859141366,
  52.982436716660835,
  44.67797974389047],
 'prec_str': '80.4/64.0/53.0/44.7',
 'ratio': 0.9940175223401853}

In [28]:
tokenizer.push_to_hub('t5-super-tiny-finetuned-noisy-en-ms', organization='mesolitica')

/home/ubuntu/.local/lib/python3.8/site-packages/huggingface_hub/utils/_deprecation.py:39: FutureWarning: Pass token='t5-super-tiny-finetuned-noisy-en-ms' as keyword args. From version 0.7 passing these as positional arguments will result in an error
  warnings.warn(
/home/ubuntu/.local/lib/python3.8/site-packages/huggingface_hub/hf_api.py:79: FutureWarning: `name` and `organization` input arguments are deprecated and will be removed in v0.7. Pass `repo_id` instead.
  warnings.warn(
/home/ubuntu/.local/lib/python3.8/site-packages/huggingface_hub/hf_api.py:596: FutureWarning: `create_repo` now takes `token` as an optional positional argument. Be sure to adapt your code!
  warnings.warn(
Cloning https://huggingface.co/mesolitica/t5-super-tiny-finetuned-noisy-en-ms into local empty directory.


Download file events.out.tfevents.1659119840.husein-MS-7D31:   0%|          | 15.6k/56.4M [00:00<?, ?B/s]

Download file pytorch_model.bin:   0%|          | 1.82k/48.4M [00:00<?, ?B/s]

Download file tf_model.h5:   0%|          | 1.81k/48.5M [00:00<?, ?B/s]

Download file events.out.tfevents.1659118188.husein-MS-7D31:   0%|          | 4.24k/17.9M [00:00<?, ?B/s]

...

Clean file events.out.tfevents.1659118087.husein-MS-7D31:   0%|          | 1.00k/17.4M [00:00<?, ?B/s]

Clean file events.out.tfevents.1659118188.husein-MS-7D31:   0%|          | 1.00k/17.9M [00:00<?, ?B/s]

Clean file events.out.tfevents.1659119840.husein-MS-7D31:   0%|          | 1.00k/56.4M [00:00<?, ?B/s]

Clean file pytorch_model.bin:   0%|          | 1.00k/48.4M [00:00<?, ?B/s]

Clean file tf_model.h5:   0%|          | 1.00k/48.5M [00:00<?, ?B/s]

In [29]:
model_gen.push_to_hub('t5-super-tiny-finetuned-noisy-en-ms', organization='mesolitica')

Upload file pytorch_model.bin:   0%|          | 4.00k/48.4M [00:00<?, ?B/s]

To https://huggingface.co/mesolitica/t5-super-tiny-finetuned-noisy-en-ms
   1c432db..af5a567  main -> main



'https://huggingface.co/mesolitica/t5-super-tiny-finetuned-noisy-en-ms/commit/af5a567b8973f9345bef6ab32d8fe66863f3cae6'

In [30]:
model_tf.push_to_hub('t5-super-tiny-finetuned-noisy-en-ms', organization='mesolitica')

Upload file tf_model.h5:   0%|          | 4.00k/48.5M [00:00<?, ?B/s]

To https://huggingface.co/mesolitica/t5-super-tiny-finetuned-noisy-en-ms
   af5a567..786766e  main -> main



'https://huggingface.co/mesolitica/t5-super-tiny-finetuned-noisy-en-ms/commit/786766e3cd6dd1ef6f5ffdc6e42566eeadc0eb65'

In [32]:
!cd t5-super-tiny-finetuned-noisy-en-ms && git pull

Already up to date.


In [33]:
!cp t5-super-tiny-noisy-en-ms/*.tfevents.* t5-super-tiny-finetuned-noisy-en-ms
!cd t5-super-tiny-finetuned-noisy-en-ms && git add . && git commit -m 'add tensorboard' && git push

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
